In [2]:
import numpy as np
import pandas as pd
from copy import copy
import sys
sys.path.append('/home/ndsviriden/MinMax94/src/utils') 
from constants import data_directory, MmxColumns
from interpolation import interpolate_mmx, create_patterns
from converters import convert_raw_to_mmx
from loaders import load_mm94_stations, select_mm94_features
from geographical import find_nearest_wmo_station, add_solar_angles, add_coordinates, add_road_id
from sklearn.ensemble import IsolationForest
import gc

In [3]:
# available for test:
# 114, 119, 302, 303, 442, 511, 307, 393, 503, 504, 516, 1838, 1896, 117

# available and clean:
# 20921, 20916, 20761, 20755, 20754, 20743, 20717, 20323
# 1921, 1911, 1899, 1831, 1826, 1813, 1808, 704, 702, 635, 628, 626, 615, 593,
# 456, 454, 435, 432, 411, 401, 309, 308, 305, 302, 239, 228, 231, 200, 158, 150, 
# 126, 119, 118, 117, 116, 115, 114, 113

# available:
# 114, 115, 116, 117, 119, 302, 303, 304, 305, 306, 307,
# 308, 309, 393, 442, 470, 471, 472, 502, 503, 504, 505,
# 506, 507, 508, 511, 512, 513, 514, 515, 516, 591, 592,
# 593, 596, 597, 599, 612, 613, 614, 615, 616, 617, 618,
# 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629,
# 836, 837, 838, 839, 862, 863, 874, 875, 888, 1820, 1832,
# 1833, 1834, 1835, 1836, 1838, 1896, 1899, 4007

# small outliers:
# 114, 119, 302, 303, 442, 511, 512, 514, 629, 838, 839

# big bugs
# 307, 393, 503, 504, 505, 516, 619, 1838, 1896 -- big bugs

## Loading data

In [4]:
%%time

test_station_id = [114, 119, 302, 303, 307, 393, 442, 511, 393, 503, 504, 516, 1838, 1896, 117] 


train_station_id = [20921, 20761, 113, 115, 1899] 
                   #1921, 1899, 1826, 1808, 702,
                    #626, 615, 456, 435, 411,
                   #308, 239, 223, 152, 150,
                   #126, 118, 116, 115, 113, 20754, 20717]

raw = load_mm94_stations(train_station_id + test_station_id)
raw = select_mm94_features(raw, ['t_air', 't_road', 't_underroad', 'pressure', 'dampness'])

mmx_rwis = convert_raw_to_mmx(raw)
mmx_rwis_interpolated = interpolate_mmx(mmx_rwis)
data = create_patterns(mmx_rwis_interpolated)

data['data_solar_azimuth'], data['data_solar_altitude'] = add_solar_angles(data)
data['data_latitude'], data['data_longitude'] = add_coordinates(data)
data['data_road'] = add_road_id(data)

del data[MmxColumns.ID_AIR_TEMPERATURE], data[MmxColumns.ID_UNDERGROUND_TEMPERATURE], \
    data[MmxColumns.ID_PRESSURE], data[MmxColumns.ID_HUMIDITY]

train = data[data['station_id'].isin(train_station_id)]
train = train.reset_index(drop=True)

test = data[data['station_id'].isin(test_station_id)]
test = test.reset_index(drop=True)

del data, raw, mmx_rwis, mmx_rwis_interpolated
gc.collect()

/home/ndsviriden/MinMax94/src/utils/geographical.py:65: RuntimeWarning: invalid value encountered in arccos
  solar_azimuth = np.arccos(cos_az) * np.sign(h_rad)


CPU times: user 33.3 s, sys: 3.2 s, total: 36.5 s
Wall time: 37.4 s


## Add labels to test

In [5]:
loaded_valid_test_ids = []
for station_id in test_station_id:
    raw = pd.read_csv('/home/ndsviriden/data_valid/' + str(station_id) + '_valid.csv', index_col=False)
    loaded_valid_test_ids.append(raw['id'])

valid_test_ids = pd.concat(loaded_valid_test_ids)
valid_test_ids = list(valid_test_ids.values.ravel())
test['label_true'] = (~test[MmxColumns.ID_ROAD_TEMPERATURE].isin(valid_test_ids)).astype(int)
# test['label_predicted'] = np.random.binomial(1, 0.15, len(test))

## Clean train data

In [ ]:
loaded_valid_train_ids = []
for station_id in train_station_id:
    raw = pd.read_csv('/media/ndsviriden/Elements/MinMax94/data/CSV/clean_ids/' + str(station_id) + '_valid.csv', index_col=False, header=None)
    loaded_valid_train_ids.append(raw)

valid_train_ids = pd.concat(loaded_valid_train_ids)
valid_train_ids = list(valid_train_ids.values.ravel())
train['clean'] = train[MmxColumns.ID_ROAD_TEMPERATURE].isin(valid_train_ids)
train = train[train['clean']]
del train['clean']

## Feature selection

In [5]:
def create_data(df):
    df_res = pd.DataFrame(index=df.index)
    df_res['date_time_utc'] = copy(df['date_time_utc'])
    df_res['station_id'] = copy(df['station_id'])
    for column in [col for col in df.columns if col.startswith('data_')]:
        #print(column)
        df_res['{0}'.format(column)] = copy(df[column])
        if column not in ('data_latitude', 'data_longitude', 
                         'data_solar_altitude', 'data_solar_azimuth', 'data_road'):
            #print(column)
            df_res['{0}_lag_1'.format(column)] = df_res['{0}'.format(column)].shift(1) 
            df_res['{0}_lag_2'.format(column)] = df_res['{0}'.format(column)].shift(2)
            df_res['{0}_lag_3'.format(column)] = df_res['{0}'.format(column)].shift(3)
            df_res['{0}_lag_4'.format(column)] = df_res['{0}'.format(column)].shift(4)
            df_res['{0}_lag_5'.format(column)] = df_res['{0}'.format(column)].shift(5)
            #df_res['{0}_lag_6'.format(column)] = df_res['{0}'.format(column)].shift(6)
            #df_res['{0}_lag_7'.format(column)] = df_res['{0}'.format(column)].shift(7)
            #df_res['{0}_lag_8'.format(column)] = df_res['{0}'.format(column)].shift(8)
        
            #if column not in ('solar_altitude', 'solar_azimuth'):
            #df_res['{0}_diff_12'.format(column)] = df_res['{0}_lag_1'.format(column)] \
            #                                        - df_res['{0}_lag_2'.format(column)]
            df_res['{0}_diff_23'.format(column)] = df_res['{0}_lag_2'.format(column)] \
                                                    - df_res['{0}_lag_3'.format(column)]
            df_res['{0}_diff_34'.format(column)] = df_res['{0}_lag_3'.format(column)] \
                                                    - df_res['{0}_lag_4'.format(column)]
            df_res['{0}_diff_45'.format(column)] = df_res['{0}_lag_4'.format(column)] \
                                                    - df_res['{0}_lag_5'.format(column)]
            #df_res['{0}_diff_56'.format(column)] = df_res['{0}_lag_5'.format(column)] \
            #                                       - df_res['{0}_lag_6'.format(column)]
            #df_res['{0}_diff_67'.format(column)] = df_res['{0}_lag_6'.format(column)] \
            #                                        - df_res['{0}_lag_7'.format(column)]
            #df_res['{0}_diff_78'.format(column)] = df_res['{0}_lag_7'.format(column)] \
            #                                        - df_res['{0}_lag_8'.format(column)]
            if column != 'data_t_road':
                del df_res['{0}'.format(column)]
    
    #df_res.set_index('date_time', inplace=True)
    df_res['data_dayofyear_cos'] = np.cos(df_res['date_time_utc'].dt.dayofyear / 365 * np.pi)

    df_res['data_dayofyear_sin'] = np.sin(df_res['date_time_utc'].dt.dayofyear / 365 * np.pi)

    df_res['data_hour_cos'] = np.cos(df_res['date_time_utc'].dt.hour / 24 * np.pi)

    df_res['data_hour_sin'] = np.sin(df_res['date_time_utc'].dt.hour / 24 * np.pi)

    df_res['data_month_cos'] = np.cos(df_res['date_time_utc'].dt.month / 12 * np.pi)

    df_res['data_month_sin'] = np.sin(df_res['date_time_utc'].dt.month / 12 * np.pi)
    df_res = df_res.fillna(-10000)
    return df_res

In [6]:
import xgboost as xgb

point = pd.Timestamp(2015, 6, 1)

X_train = create_data(train[train['date_time_utc'] < point])
features = [col for col in X_train if col.startswith('data_') and (col!='data_t_road')]
X_train = X_train[features].values
y_train = train.loc[train['date_time_utc'] < point, 'data_t_road'].values.reshape(-1, 1)

X_valid = create_data(train[train['date_time_utc'] >= point])
X_valid = X_valid[features].values
y_valid = train.loc[train['date_time_utc'] >= point, 'data_t_road'].values.reshape(-1, 1)

X_test = create_data(test)
X_test = X_test[features].values
y_test = test['data_t_road'].values.reshape(-1, 1)


dtrain = xgb.DMatrix(X_train, y_train)
dval = xgb.DMatrix(X_valid, y_valid)
dtest = xgb.DMatrix(X_test, y_test)

## Algorithm

In [7]:
from sklearn.metrics import mean_squared_error as mse
import xgboost as xgb
from functools import partial

params = {}

params["eval_metric"] = 'mae'
params["eta"] = 0.05
params["lambda"] = 0.3
params["subsample"] = 0.8 #1
params["min_child_weight"] = 1.
params["colsample_bytree"] = 0.5
params["max_depth"] = 9
params["silent"] = 1
params["gamma"] = 0.1
delta = 1.
#delta = 1.
watchlist = [(dtrain, 'train'), (dval, 'valid')]

#-----------------------------------------------------------------------------------------------
print('-----------MEDIAN-----------')
clf_dirty = xgb.train(params, dtrain, 200, watchlist, verbose_eval=20)

-----------MEDIAN-----------
[0]	train-mae:nan	valid-mae:nan
[20]	train-mae:nan	valid-mae:nan
[40]	train-mae:nan	valid-mae:nan
[60]	train-mae:nan	valid-mae:nan
[80]	train-mae:nan	valid-mae:nan
[100]	train-mae:nan	valid-mae:nan
[120]	train-mae:nan	valid-mae:nan
[140]	train-mae:nan	valid-mae:nan
[160]	train-mae:nan	valid-mae:nan
[180]	train-mae:nan	valid-mae:nan
[199]	train-mae:nan	valid-mae:nan


In [ ]:
y_true = dtest.get_label()
y_pred = clf_dirty.predict(dtest).ravel()
upper = y_pred + 0.48 * 10
lower = y_pred - 0.48 * 10
test['label_predicted'] = ((y_true > upper) | (y_true < lower))

In [ ]:
len(test[test['label_predicted']==1]) / len(test)

## Calculate Precision

In [ ]:
true_anomalies_ids = set()
window = pd.Timedelta('3h')

st_id = [114, 119, 302, 303, 442, 511, 504, 516, 1838, 1896, 117]
for station in st_id:
    df = test[test['station_id']==station]
    true_anomalies = df[df['label_true']==1]
    for anomaly in true_anomalies.iterrows():
        dt = anomaly[1]['date_time_utc']
        locality = test[(test['date_time_utc'] >= (dt - window)) & 
                       (test['date_time_utc'] <= (dt + window))]
        
        true_anomalies_ids.update(set(locality.index))
        # print(list(locality.index))

predicted_anomalies_ids = set(test[((test['label_predicted']==1) & (test['station_id'].isin(st_id)))].index)

tp = set.intersection(true_anomalies_ids, predicted_anomalies_ids)
precision = len(tp) / len(predicted_anomalies_ids)
precision

## Calculate Recall

In [ ]:
predicted_anomalies_ids = set()
window = pd.Timedelta('3h')

# [114, 119, 302, 303, 442, 511, ?307, ?393, ?503, 504, 516, 1838, 1896, 117]
# for station in test.station_id.unique():
st_id = [114, 119, 302, 303, 442, 511, 504, 516, 1838, 1896, 117]
for station in st_id:
    df = test[test['station_id']==station]
    predicted_anomalies = df[df['label_predicted']==1]
    for anomaly in predicted_anomalies.iterrows():
        dt = anomaly[1]['date_time_utc']
        locality = test[(test['date_time_utc'] >= (dt - window)) & 
                       (test['date_time_utc'] <= (dt + window))]
        
        predicted_anomalies_ids.update(set(locality.index))
        # print(list(locality.index))

true_anomalies_ids = set(test[(test['label_true']==1) & (test['station_id'].isin(st_id))].index)

tp = set.intersection(true_anomalies_ids, predicted_anomalies_ids)
recall = len(tp) / len(true_anomalies_ids)
recall, len(true_anomalies_ids)

In [22]:
import matplotlib.pyplot as plt


# test_station_id = [114, 119, 302, 303, 307, 393, 442, 511, 393, 503, 504, 516, 1838, 1896, 117] 

# 307, 393, 503, 504, 505, 516, 619, 1838, 1896 -- big bugs

df_station = copy(train[train['station_id']==1899].set_index('date_time_utc'))
#df_station = copy(train[train['station_id']==113].set_index('date_time_utc'))

start = pd.Timestamp(2012, 1, 1)
end = pd.Timestamp(2014, 2, 1)

to_plot = df_station[(df_station.index<=end) & (df_station.index>=start)]

plt.figure(figsize=(40, 10))

#cond = (to_plot['label_true']) == 1
#plt.plot_date(to_plot[cond].index, to_plot[cond]['data_t_road'], 'g',
#                      linestyle='none', marker='o', markersize=12, label='t_road')



"""for elem in [True, False]:
    cond = (to_plot['label_predicted']) != elem
    if elem:
        i=1
        plt.plot_date(to_plot[cond].index, to_plot[cond]['data_t_road'], 'b',
                      linestyle='none', marker='o', markersize=8, label='t_road')
    else:
        plt.plot_date(to_plot[cond].index, to_plot[cond]['data_t_road'], 'r.',
                              linestyle='none', marker='o', label='outliers', markersize=8)"""

plt.plot_date(to_plot.index, to_plot['data_t_road'], 'b',
                      linestyle='-', marker='None', markersize=8, label='t_road', linewidth=3)

plt.grid()
plt.legend(fontsize=40)
plt.ylabel(r' Road temperature, $^{\circ}C$', fontsize=40)

plt.tick_params(labelsize=40)
#plt.show()
plt.savefig('/home/ndsviriden/PycharmProjects/MinMax94/src/pre_defence_plots/typical_large')